## Языковое моделирование

Языковое моделирование заключается в приписывании вероятности последовательности слов. Сейчас языковые модели используются практически во всех nlp задачах. Всякие Берты и Элмо - языковые модели. 

Это достаточно сложная тема, поэтому будем разбирать постепенно. Сегодня разберём самые основы. Научимся приписывать вероятность последовательности слов и попробуем генерировать текст.

Возьмем два разных корпуса: новостной и сообщения с vk.

In [70]:
vk = open('vk_texts_norm.txt').read()
news = open('lenta2.txt').read()

По длине оно сопоставимы.

In [71]:
print("Длина Вк ----", len(vk))
print("Длина Лента -", len(news))


Длина Вк ---- 5497266
Длина Лента - 5897301


Напишем простую функцию для нормализации. 

In [73]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text


Сравним тексты по токенам

In [74]:
norm_vk = normalize(vk)
norm_news = normalize(news)

In [75]:
print("Длина корпуса ВКонтакте текстов в токенах - ", len(norm_vk))
print("Длина корпуса новостных текстов в токенах - ", len(norm_news))

Длина корпуса ВКонтакте текстов в токенах -  848722
Длина корпуса новостных текстов в токенах -  766869


И по уникальным токенам

In [76]:
print("Уникальных токенов в корпусе ВКонтакте - ", len(set(norm_vk)))
print("Уникальный токенов в корпусе Ленты - ", len(set(norm_news)))

Уникальных токенов в корпусе ВКонтакте -  114023
Уникальный токенов в корпусе Ленты -  79054


Посчитаем, сколько раз встречаются слова и выведем самые частотные.

In [77]:
from collections import Counter

vocab_vk = Counter(norm_vk)
vocab_news = Counter(norm_news)


In [78]:
vocab_vk.most_common(10)

[('и', 32672),
 ('в', 21946),
 ('не', 13763),
 ('на', 10688),
 ('с', 8435),
 ('что', 7473),
 ('я', 7424),
 ('а', 5129),
 ('это', 4768),
 ('как', 4621)]

In [81]:
vocab_news.most_common(10)

[('в', 35997),
 ('и', 17120),
 ('на', 14306),
 ('по', 10053),
 ('что', 9136),
 ('с', 8146),
 ('не', 6612),
 ('как', 4126),
 ('о', 4047),
 ('из', 3892)]

Сравнивать употребимость конкретных слов в разных текстах в абсолютных числах неудобно. Нормализуем счётчики на размеры текстов. Так у нас получается вероятность слова.

In [80]:
# Вк
probas_vk = Counter({word:c/len(norm_vk) for word, c in vocab_vk.items()})
probas_vk.most_common(20)

[('и', 0.038495526214708704),
 ('в', 0.02585770134390295),
 ('не', 0.01621614615857725),
 ('на', 0.012593051670629487),
 ('с', 0.009938472197020933),
 ('что', 0.008805003287295486),
 ('я', 0.008747269423910303),
 ('а', 0.006043203781685876),
 ('это', 0.005617858380011358),
 ('как', 0.005444656789855807),
 ('для', 0.004745959218684092),
 ('по', 0.004610461376045395),
 ('https', 0.0041120649635569715),
 ('к', 0.00406964824760051),
 ('за', 0.0038917336890053516),
 ('ты', 0.0038434257624993815),
 ('от', 0.0038363563098399714),
 ('но', 0.003751522877927048),
 ('»', 0.003657263509134911),
 ('«', 0.003625450972167565)]

In [83]:
# Лента
probas_news = Counter({word:c/len(norm_news) for word, c in vocab_news.items()})
probas_news.most_common(20)

[('в', 0.04694022055918286),
 ('и', 0.022324543044509558),
 ('на', 0.01865507668193655),
 ('по', 0.013109149020236834),
 ('что', 0.011913377643378464),
 ('с', 0.010622413997697129),
 ('не', 0.008622072348732314),
 ('как', 0.005380319194021404),
 ('о', 0.005277302903103399),
 ('из', 0.005075182332314907),
 ('к', 0.004138907688275312),
 ('за', 0.0038911469885990957),
 ('россии', 0.003790738705046103),
 ('для', 0.003450393743911933),
 ('его', 0.003189593007410653),
 ('от', 0.0031817689853156144),
 ('он', 0.003166120941125538),
 ('сообщает', 0.003026592547097353),
 ('а', 0.002902712197259245),
 ('также', 0.0027149356669783236)]

Попробуем понять, какое предложение относится к Вк, а какое к новостям, если судить по униграммам

In [84]:
phrase = 'Технические возможности устаревшего российского судна не позволили разгрузить его у терминала'

prob = Counter({'news':0, 'vk':0})

for word in normalize(phrase):
    prob['news'] += probas_news.get(word, 0)
    prob['vk'] += probas_vk.get(word, 0)



In [85]:
prob.most_common()

[('vk', 0.021858747622896545), ('news', 0.014115839863131772)]

In [88]:
phrase = 'Четыре сотрудника саратовского УФСИН уволены после сообщений о пытках'

prob = Counter({'news':0, 'vk':0})

for word in normalize(phrase):
    prob['news'] += probas_news.get(word, 0)
    prob['vk'] += probas_vk.get(word, 0)



In [89]:
prob.most_common()

[('news', 0.007285468574163254), ('vk', 0.003784513657004296)]

Результаты получаются не очень точные. Возможно это из-за того, что мы считаем слова независимыми друг от друга. А это очевидно не так

По-хорошему вероятность последовательности нужно расчитывать по формуле полной вероятности. Но у нас не очень большие тексты и мы не можем получить вероятности для длинных фраз (их просто может не быть в текстах). Поэтому мы воспользуемся предположением Маркова и будем учитывать только предыдущее слово.

Чтобы расчитать вероятность с таким предположением, нам достаточно найти количество вхождений для каждого биграмма.

In [90]:
from nltk import ngrams

def ngrammer(tokens, n=2):
    return ngrams(tokens, 2)

Для того, чтобы у нас получились честные вероятности и можно было посчитать вероятность первого слова, нам нужно добавить тэг маркирующий начало предложений \< start \>

Дальше мы попробуем сгенерировать текст, используя эти вероятности, и нам нужно будет когда-то остановится. Для этого добавим тэг окончания \< end \>

Ну и поделим все на предложения

In [94]:
from nltk.tokenize import sent_tokenize

sentences_news = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news)]
sentences_vk = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(vk)]

In [95]:
unigrams_news = Counter()
bigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))

    
unigrams_vk = Counter()
bigrams_vk = Counter()

for sentence in sentences_vk:
    unigrams_vk.update(sentence)
    bigrams_vk.update(ngrammer(sentence))

In [96]:
bigrams_news.most_common(10)

[(('<start>', 'в'), 3831),
 (('<start>', 'по'), 3043),
 (('<start>', 'как'), 2064),
 (('риа', 'новости'), 1669),
 (('по', 'словам'), 966),
 (('что', 'в'), 888),
 (('об', 'этом'), 871),
 (('<start>', 'однако'), 845),
 (('как', 'сообщает'), 809),
 (('<start>', 'на'), 805)]

In [97]:
bigrams_vk.most_common(10)

[(('https', 'vk'), 2647),
 (('vk', 'com'), 2389),
 (('<start>', 'и'), 2209),
 (('<start>', 'я'), 1630),
 (('<start>', 'в'), 1611),
 (('<start>', 'а'), 1272),
 (('и', 'не'), 1164),
 (('╬═╬', '╬═╬'), 1154),
 (('<start>', 'но'), 887),
 (('<start>', 'не'), 845)]

Чтобы посчитать условную вероятность мы можем поделить количество вхождений на количество вхождений первого слова.

In [105]:
# phrase = 'Технические возможности устаревшего российского судна не позволили разгрузить его у терминала'
# phrase = 'Ныть надо меньше и работать больше.'
# phrase = 'напиши мне в лс тогда и отвечу'
phrase = 'Четыре сотрудника саратовского УФСИН уволены после сообщений о пытках'
prob = Counter()
for ngram in ngrammer(['<start>'] + normalize(phrase) + ['<end>']):
    word1, word2 = ngram[0], ngram[1]
    
    if word1 in unigrams_vk and ngram in bigrams_vk:
        prob['vk'] += np.log(bigrams_vk[ngram]/unigrams_vk[word1])
    else:
        prob['vk'] += np.log(0.001)
    
    if word1 in unigrams_news and ngram in bigrams_news:
        prob['news'] += np.log(bigrams_news[ngram]/unigrams_news[word1])
    else:
        prob['news'] += np.log(0.001)

prob['news'] = np.exp(prob['news'])
prob['vk'] = np.exp(prob['vk'])

In [106]:
prob.most_common()

[('news', 1.0764269200870128e-26), ('vk', 5.121638924455876e-32)]

Работает получше. Мы воспользовались небольшим хаком - для слов или биграммов, которых не было у нас в словаре, прибавляли низкую вероятность. Надо было делать сглаживания по правилам, но это долго и сложно...

Проблем с неизвестными словами у нас не будет, если мы будем пытаться сгенерировать новый текст. Давайте попробуем это сделать.

In [107]:
matrix_vk = np.zeros((len(unigrams_vk), 
                   len(unigrams_vk)))
id2word_vk = list(unigrams_vk)
word2id_vk = {word:i for i, word in enumerate(id2word_vk)}


for ngram in bigrams_vk:
    word1, word2 = ngram[0], ngram[1]
    matrix_vk[word2id_vk[word1]][word2id_vk[word2]] =  (bigrams_vk[ngram]/
                                                                     unigrams_vk[word1])



In [108]:
# создадим матрицу вероятностей перейти из одного слова в другое
matrix_news = np.zeros((len(unigrams_news), 
                   len(unigrams_news)))

id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}


# вероятность расчитываем точно также
for ngram in bigrams_news:
    word1, word2 = ngram[0], ngram[1]
    matrix_news[word2id_news[word1]][word2id_news[word2]] =  (bigrams_news[ngram]/
                                                                     unigrams_news[word1])



Для генерации нам понадобится функция np.random.choice , которая выбирает случайный объект из заданных. Ещё в неё можно подать вероятность каждого объекта и она будет доставать по ним (не только максимальный по вероятности)

In [109]:

def generate(matrix, id2word, word2id, n=100, start='<start>'):
    text = []
    current_idx = word2id[start]
    
    for i in range(n):
        
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            chosen = word2id['<start>']
        current_idx = chosen
    
    return ' '.join(text)

In [112]:
print(generate(matrix_vk, id2word_vk, word2id_vk).replace('<end>', '\n'))

святой 
 ❗ 💎 открыть для бизнеса классные летние прогулочные коляски авторские методики разработки ижевских оружейников из нее такой подарок в вашем грузовом отсеке побывал в lol 
 – тех 
 думайте письменно пообещал что где-то или пуститься в игре целуй и сверх y многому другому 
 она самый неадекват 
 планировалось назначить женщину обманули предпочли а всё это в ее потерял веру в клубе легко если где-нибудь глубоко в своей маме и kollontai 
 любите друг друга прямо сейчас думаю и делении дробей ⠀ 🆘скажите 👉🏻вам когда деньги не смеяться и понимает 
 начни действовать автоматически только если вы


In [113]:
print(generate(matrix_news, id2word_news, word2id_news).replace('<end>', '\n'))

в доме № 783 
 лидеры стран которые запрещают хранение на заседании совета министров финансов фбр и оформления документов 
 автомобиль lincoln navigator и врезался мини-трактор обслуживавший заправку лайнера погибли 9 кандидатов от эксимбанка сша семьей не пришло подчеркнул что бин ладеном и авторитета михаила леонтьева и все ведущие разработку еще на металлолом 
 во вторник предъявлено так и распространении коммерческих запусков космических аппаратов для партии консерваторов находящаяся в служебной проверки все результаты переговоров с показом 30 рублей право опротестовать вчерашнее решение таганрогской окружной избирательной комиссии проект модернизации почти парализована 
 позже агентство итар-тасс высокопоставленный сотрудник госбезопасности и на каждую


Попробуйте сделать триграммную модель на основе кода выше.

Подсказки:
    - нужно будет добавить еще один тэг <start>
    - еще одна матрица не нужна, можно по строкам хранить биграмы, а по колонкам униграммы
    - тексты должны быть очень похоже на нормальные (если у вас получается рандомная каша, вы что-то делаете не так)